In [1]:
from bokeh.io import output_notebook, push_notebook
from bokeh.models import BoxSelectTool, WheelZoomTool, PanTool, BoxZoomTool, TapTool, CustomJS, ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.layouts import column

from ipywidgets.widgets import Button, HBox
from IPython.display import display

output_notebook()

Loading BokehJS ...

In [ ]:
k = 100
padding = 50
i = 0
idx = list()

set_button = Button(description = "Set")
unset_button = Button(description = "Unset")

prev_button = Button(description = 'Previous')
next_button = Button(description = 'Next')

def on_click_set(b):    
    y_true[np.array(idx)] = 1    
    source = ColumnDataSource(
        data = dict(
            x = np.arange(k), 
            meas = Ym[:k],
            comp = Yc[:k],
            anomaly = y_true[:k]
        )
    )
    
def on_click_unset(b):   
    y_true[np.array(idx)] = 0    
    source = ColumnDataSource(
        data = dict(
            x = np.arange(k), 
            meas = Ym[i-padding : i + k + padding],
            comp = Yc[i-padding : i + k + padding],
            anomaly = y_true[i-padding : i + k + padding]
        )
    )

    
def on_click_unset(b):   
    y_true[np.array(idx)] = 0    
    source = ColumnDataSource(
        data = dict(
            x = np.arange(k), 
            meas = Ym[i-padding : i + k + padding],
            comp = Yc[i-padding : i + k + padding],
            anomaly = y_true[i-padding : i + k + padding]
        )
    )
    
set_button.on_click(on_click_set)
unset_button.on_click(on_click_unset)
    

gh_tools = [BoxSelectTool(dimensions=['width']), 
            TapTool(),
            BoxZoomTool(dimensions=['width']),
            BoxZoomTool(dimensions=['height']),
            WheelZoomTool(dimensions=['width']),
            WheelZoomTool(dimensions=['height']),
            PanTool()]

gh_plot = figure(title = 'test', 
                 plot_height = 300, 
                 plot_width = 900, 
                 y_range = (-1,4), 
                 x_range = (0,k),
                 tools = gh_tools,
                 toolbar_location = "above")

source = ColumnDataSource(
    data = dict(
        x = np.arange(k), 
        meas = Ym[i:i+k],
        comp = Yc[i:i+k],
        anomaly = y_true[i:i+k]
    )
)

select_cb = CustomJS(args = dict(gh_plot = gh_plot), 
                     code = """var inds = cb_obj.get('selected')['1d'].indices;
                               var d1 = cb_obj.get('data');
                               console.log(d1)
                               var kernel = IPython.notebook.kernel;
                               IPython.notebook.kernel.execute("idx = " + inds);"""
                    )
source.callback = select_cb

gh_plot.line  ("x", "meas", source = source, color = '#ff0000', line_width = 1)
gh_plot.circle("x", "meas", source = source, color = '#ff0000', size = 2)

gh_plot.line  ("x", "comp", source = source, color = '#0000ff', line_width = 1)
gh_plot.circle("x", "comp", source = source, color = '#0000ff', size = 2)


anomaly_tools = [BoxSelectTool(), 
                 TapTool(),
                 BoxZoomTool(dimensions=['width']),
                 WheelZoomTool(dimensions=['width']),
                 PanTool(dimensions=['width'])]

anomaly_plot = figure(title = 'test', 
                      plot_height = 300, 
                      plot_width = 900, 
                      y_range = (-0.5,1.5), 
                      x_range = gh_plot.x_range,
                      tools = anomaly_tools,
                      toolbar_location = "above")

anomaly_plot.line  ("x", "anomaly", source = source, color = '#000000', line_width = 1)
anomaly_plot.circle("x", "anomaly", source = source, color = '#000000', size = 2)

In [ ]:
display(HBox([set_button, unset_button]))
show(column(gh_plot, anomaly_plot))